In [10]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import glob

In [11]:
# read in the data frame, drop the unnecessary index row & transpose it so rows are loci and columns are individuals.
df = pd.read_parquet("~/Documents/UMontana/Research/YNP/AHQsd/AHQsd_analyses/AHQsd_genotypes_filt.parquet")

In [12]:
df = df.reset_index()

In [13]:
# label by what linkage group the loci belong to
tmpDF = pd.DataFrame(columns=['chr','site'])
tmpDF[['chr','site']] = df['index'].str.split('_', expand=True)

In [14]:
df['chr'] = tmpDF['chr']

In [15]:
df['site'] = tmpDF['site']

In [16]:
# split them
for chromo in range(1,15):
    lg_chromo = df[df.chr == str(chromo)]
    lg_chromo.to_csv(f'data/lg_chromo_{chromo}.csv',index=False)

In [17]:
tdf = df.T

In [29]:
def list_name_indivs(ds):
    return ds.columns.to_list()[2:323] 

def create_bins(subset_ds):
    # create bins of 18 SNPs each in each linkage group
    bins = [i for i in np.arange(subset_ds.index.min(), subset_ds.index.max(), 18)] + [np.inf]
    return bins

def create_stats_template(ds, chromo_number):
    # create dataframes for the loop to write to. 
    return pd.DataFrame(columns=["indv_name","count","mean",f"num_of_nans_chr{chromo_number}"])

def cut_by_bins(subset_ds, bins):
    return pd.cut(subset_ds.index,bins=bins)

def create_stats_dataframe(small_list, ds, bins, chromo_number):
    stats_dataframe = create_stats_template(ds, chromo_number)
    chromo_number = ds['chr'].iloc[0]
    for col in small_list:
        mdf = ds[['index',col]]
        grouped_df = mdf.groupby(bins)[col].agg(['mean','count'])
        grouped_df[f"num_of_nans_chr{chromo_number}"] = 25 - grouped_df["count"]
        grouped_df["indv_name"] = col
        stats_dataframe  = stats_dataframe.append(grouped_df)
    return stats_dataframe

def remove_high_NaNs(stats_dataframe, chromo_number):
    stats_dataframe.loc[stats_dataframe[f"num_of_nans_chr{chromo_number}"] > 9, 'mean'] = np.nan
    return stats_dataframe

def narrow(filt_stats_dataframe): 
    filt_stats_narrow = filt_stats_dataframe[['indv_name', 'mean']]
    filt_stats_narrow['bin'] = filt_stats_narrow.index
    filt_stats_wide = filt_stats_narrow.pivot_table(index='bin', columns='indv_name', values='mean', aggfunc='first', dropna=False)
    return filt_stats_wide

#def cleanup_cols_names(df):
    #df.columns = df.columns.str.split('processed/').str[1].tolist()   
    #return df

def create_bin_template():
    col_names = ['level_0', 'first','last']
    return pd.DataFrame(columns=col_names)

def create_append_template():
    col_names = df.columns[2:323]
    return pd.DataFrame(columns=col_names)

def convert_to_genotypes(template_df):
    choiceList = ['BB', 'AA', 'AB']
    all_indvs = template_df.columns
    for i in all_indvs:
        condList = [(template_df[i] > 1.4), (template_df[i] < .6), ((template_df[i] < 1.3) & (template_df[i] > .7))]
        template_df[i] = np.select(condList, choiceList)
    for i in template_df.columns:
        template_df[i].replace('0',np.nan,inplace=True)
    return template_df

def rename_SNP_bins(ds):
    bin_rename = pd.DataFrame(ds['index'].groupby(cut_bins).agg(['first', 'last']).stack())
    bin_rename = bin_rename.reset_index()
    bin_rename = bin_rename.pivot_table(index='level_0', columns='level_1', values =0,aggfunc='first', dropna=False)
    bin_rename = bin_rename.reset_index()
    return bin_rename


In [30]:
subset_file_list = glob.glob('data/*.csv')
ind_list = list_name_indivs(df)

template_df = create_append_template()
bin_df = create_bin_template()

for fil in subset_file_list:
    print(fil)
    #load subset of dataset that was subset by chromosome
    ds = pd.read_csv(fil)
    chromo_number = ds['chr'].iloc[0]
    #create bins
    bins = create_bins(ds)
    #create empty stats template
    stats_template_df = create_stats_template(ds, chromo_number)
    # cut each linkage group into bins of X SNPs each.
    cut_bins = cut_by_bins(ds, bins)
    # start renaming SNP bins
    SNP_bin_rename = rename_SNP_bins(ds)
    # create dataframe of stats
    stats_dataframe = create_stats_dataframe(ind_list, ds, cut_bins, chromo_number)
    # remove dfs with lots of NAs 
    filt_stats_dataframe = remove_high_NaNs(stats_dataframe, chromo_number)
    # pivot magic
    filt_stats_wide = narrow(filt_stats_dataframe)
    #cleanup col names
    #clean_df = cleanup_cols_names(filt_stats_wide)
    template_df = template_df.append(filt_stats_wide,ignore_index=True)
    bin_df = bin_df.append(SNP_bin_rename, ignore_index=True)
#     break
#     cleaned_ds.to_csv('final_data')

data/lg_chromo_9.csv


<ipython-input-29-baa4727b9fb1>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filt_stats_narrow['bin'] = filt_stats_narrow.index


data/lg_chromo_8.csv
data/lg_chromo_5.csv
data/lg_chromo_14.csv
data/lg_chromo_4.csv
data/lg_chromo_6.csv
data/lg_chromo_7.csv
data/lg_chromo_3.csv
data/lg_chromo_12.csv
data/lg_chromo_13.csv
data/lg_chromo_2.csv
data/lg_chromo_11.csv
data/lg_chromo_10.csv
data/lg_chromo_1.csv


In [31]:
# label the bins by the means of their SNP positions

tmpDF1 = pd.DataFrame(columns=['chr','site1', 'chr_', 'site2'])
tmpDF1[['chr','site1']] = bin_df['first'].str.split('_', expand=True)
tmpDF1[['chr_','site2']] = bin_df['last'].str.split('_', expand=True)

tmpDF1['sum'] = pd.to_numeric(tmpDF1['site1']) + pd.to_numeric(tmpDF1['site2'])

tmpDF1['mean'] = tmpDF1['sum'] / 2

tmpDF1['bin_mean'] = tmpDF1['chr']+"_"+tmpDF1['mean'].astype(str)

#add the means to the whole spreadsheet
template_df.insert(loc=0, column='bin_mean', value=tmpDF1['bin_mean'])


In [32]:
# add a chr column
template_df.insert(loc=0, column='chr', value=tmpDF1['chr'])

In [33]:
# add a pos column
template_df.insert(loc=0, column='pos', value=tmpDF1['mean'])

In [34]:
#i have figured out almost everything except how to replace the values w/AA AB BB, 
# and I know there is a way to do it without for loops but my brain can't do it right now
template_df.to_parquet("AHQsd_F2_SNPs_windowed.parquet")

In [35]:
td = pd.read_parquet("AHQsd_F2_SNPs_windowed.parquet")

In [36]:
td_geno = convert_to_genotypes(td.iloc[:, 3:324])

In [37]:
td_geno.insert(loc=0, column='bin_mean', value=td['bin_mean'])
td_geno.insert(loc=0, column='chr', value=td['chr'])
td_geno.insert(loc=0, column='pos', value=td['pos'])

In [27]:
td_geno.to_parquet("AHQsd_F2_genotypes_windowed.parquet")

In [28]:
td_geno.to_csv("AHQsd_F2_genotypes_windowed.csv")

In [38]:
td_geno

,pos,chr,bin_mean,AHQsd_5.01C,AHQsd_5.01D,AHQsd_5.01E,AHQsd_5.01F,AHQsd_5.01G,AHQsd_5.01H,AHQsd_5.02A,...,AHQsd_3.10H,AHQsd_3.11B,AHQsd_3.11C,AHQsd_3.11D,AHQsd_3.11E,AHQsd_3.11G,AHQsd_3.11H,AHQsd_3.12D,AHQsd_3.12E,AHQsd_3.12G
0,94759.0,9,9_94759.0,BB,NaN,NaN,NaN,AA,AA,AA,...,NaN,NaN,AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,124989.0,9,9_124989.0,BB,NaN,NaN,AB,AA,AA,AA,...,NaN,NaN,AA,NaN,AA,NaN,NaN,NaN,NaN,NaN
2,257994.5,9,9_257994.5,NaN,NaN,NaN,NaN,NaN,AB,NaN,...,NaN,NaN,AB,NaN,AB,NaN,NaN,NaN,NaN,NaN
3,675849.5,9,9_675849.5,NaN,NaN,NaN,NaN,NaN,AA,NaN,...,NaN,NaN,AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1052261.0,9,9_1052261.0,AB,NaN,NaN,NaN,AB,AB,AB,...,NaN,AB,AB,NaN,AB,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,11039990.5,1,1_11039990.5,AA,NaN,NaN,NaN,NaN,AA,NaN,...,NaN,NaN,NaN,NaN,AA,NaN,NaN,NaN,NaN,NaN
1432,11086062.0,1,1_11086062.0,AB,NaN,NaN,NaN,NaN,NaN,AB,...,NaN,AB,NaN,NaN,AB,AA,NaN,NaN,NaN,NaN
1433,11305685.0,1,1_11305685.0,AB,NaN,NaN,NaN,NaN,AA,AB,...,AB,NaN,AA,NaN,AB,NaN,NaN,NaN,NaN,NaN
1434,11661804.0,1,1_11661804.0,AA,NaN,NaN,NaN,AA,AA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# use apply to make a table with the lists of genotype counts per individual
td_geno_counts_inds = td_geno.iloc[:,3:324].apply(pd.Series.value_counts, axis=0).T

# use apply to make a table with the lists of genotype counts per site
td_geno_counts_sites = td_geno.iloc[:,3:324].apply(pd.Series.value_counts, axis=1)

In [56]:
td_geno_counts_sites.insert(loc=0, column='bin_mean', value=td_geno['bin_mean'])

In [74]:
site_ratios = pd.DataFrame(columns=['AA','AB','BB'])

In [91]:
site_ratios['AA'] = np.divide(td_geno_counts_sites['AA'],
                              td_geno_counts_sites['total'])
site_ratios['AB'] = np.divide(td_geno_counts_sites['AB'],
                              td_geno_counts_sites['total'])
site_ratios['BB'] = np.divide(td_geno_counts_sites['BB'],
                              td_geno_counts_sites['total'])

In [89]:
#site_ratios.insert(loc=0, column='bin_mean', value=td_geno['bin_mean'])
site_ratios.insert(loc=0, column='pos', value=td_geno['pos'])
site_ratios.insert(loc=0, column='chr', value=td_geno['chr'])

ValueError: cannot insert pos, already exists

In [85]:
td_geno_counts_sites.sum(axis=1)

0       172.0
1       435.0
2       312.0
3        86.0
4       262.0
        ...  
1431    122.0
1432    164.0
1433    134.0
1434    186.0
1435      0.0
Length: 1436, dtype: float64

In [92]:
site_ratios

,chr,pos,bin_mean,AA,AB,BB
0,9,94759.0,9_94759.0,0.651163,0.255814,0.093023
1,9,124989.0,9_124989.0,0.310345,0.131034,0.058621
2,9,257994.5,9_257994.5,0.004808,0.495192,NaN
3,9,675849.5,9_675849.5,0.488372,0.511628,NaN
4,9,1052261.0,9_1052261.0,0.015267,0.984733,NaN
...,...,...,...,...,...,...
1431,1,11039990.5,1_11039990.5,0.311475,0.688525,NaN
1432,1,11086062.0,1_11086062.0,0.353659,0.621951,0.024390
1433,1,11305685.0,1_11305685.0,0.119403,0.850746,0.029851
1434,1,11661804.0,1_11661804.0,0.924731,0.075269,NaN


In [87]:
site_ratios

,chr,pos,bin_mean,AA,AB,BB
0,9,94759.0,9_94759.0,0.325581,0.127907,0.046512
1,9,124989.0,9_124989.0,0.206897,0.087356,0.039080
2,9,257994.5,9_257994.5,0.003205,0.330128,NaN
3,9,675849.5,9_675849.5,0.244186,0.255814,NaN
4,9,1052261.0,9_1052261.0,0.007634,0.492366,NaN
...,...,...,...,...,...,...
1431,1,11039990.5,1_11039990.5,0.155738,0.344262,NaN
1432,1,11086062.0,1_11086062.0,0.176829,0.310976,0.012195
1433,1,11305685.0,1_11305685.0,0.059701,0.425373,0.014925
1434,1,11661804.0,1_11661804.0,0.462366,0.037634,NaN
